In [0]:
# Create Gold schema
spark.sql("CREATE SCHEMA IF NOT EXISTS main.instacart_gold")


In [0]:
# Create Dimension: dim_products (Rich Dimension)
from pyspark.sql.functions import col

products = spark.table("main.instacart_silver.products")
departments = spark.table("main.instacart_silver.departments")
aisles = spark.table("main.instacart_silver.aisles")

dim_products = products \
    .join(departments, on="department_id", how="left") \
    .join(aisles, on="aisle_id", how="left") \
    .select(
        col("product_id"),
        col("product_name"),
        col("department_id"),
        col("department"),
        col("aisle_id"),
        col("aisle")
    )


In [0]:
# Save dim_products (Gold Table)
dim_products.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("main.instacart_gold.dim_products")

In [0]:
# Validate
spark.sql("SELECT COUNT(*) FROM main.instacart_gold.dim_products").show()
spark.table("main.instacart_gold.dim_products").display()


In [0]:
# Read Silver Tables
order_items = spark.table("main.instacart_silver.order_items")
orders = spark.table("main.instacart_silver.orders")


In [0]:
# Create Fact Table (Join Transformation)
from pyspark.sql.functions import col

fact_order_items = order_items \
    .join(orders, on="order_id", how="inner") \
    .select(
        col("order_id"),
        col("user_id"),
        col("product_id"),
        col("add_to_cart_order"),
        col("reordered"),
        col("order_number"),
        col("order_dow"),
        col("order_hour_of_day"),
        col("days_since_prior_order")
    )

In [0]:
# Save Gold Fact Table
fact_order_items.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("main.instacart_gold.fact_order_items")


In [0]:
# Validate
spark.sql("SELECT COUNT(*) FROM main.instacart_gold.fact_order_items").show()


In [0]:
# Business KPI Table (Top Products)
from pyspark.sql.functions import count, col

fact = spark.table("main.instacart_gold.fact_order_items")
products = spark.table("main.instacart_gold.dim_products")

top_products = fact \
    .groupBy("product_id") \
    .agg(count("*").alias("total_orders")) \
    .join(products, on="product_id", how="left") \
    .orderBy(col("total_orders").desc())


In [0]:
# Save KPI Table
top_products.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("main.instacart_gold.kpi_top_products")

In [0]:
# Validate
spark.table("main.instacart_gold.kpi_top_products").show(10)


In [0]:
# Reorder Rate KPI Table
from pyspark.sql.functions import sum as spark_sum, count, col, round

fact = spark.table("main.instacart_gold.fact_order_items")
products = spark.table("main.instacart_gold.dim_products")

reorder_kpi = fact.groupBy("product_id").agg(
    count("*").alias("total_orders"),
    spark_sum("reordered").alias("total_reorders")
)

reorder_kpi = reorder_kpi.withColumn(
    "reorder_rate",
    round(col("total_reorders") / col("total_orders"), 4)
)

reorder_kpi = reorder_kpi.join(products, on="product_id", how="left") \
    .orderBy(col("reorder_rate").desc())


In [0]:
# Save
reorder_kpi.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("main.instacart_gold.kpi_reorder_rate")


In [0]:
# Validate
spark.table("main.instacart_gold.kpi_reorder_rate").show(10)